In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# need to have all the 155 images labeled
# export that in PASCAL VOC XML

# Then, do train/test split (train=80%, validate=10%, test=10%)
# Pick a neural network (ResNet 50)
# Apply transfer learning (freeze base layers and train new fully-connected layer(s))